In [4]:
import pandas as pd
import Read_file as rf
import datetime as dt
import Dn_dt as dn


c:\Users\HP\OneDrive\Code_Play\Lib\site-packages\yfinance\base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [5]:
data = rf.Read_xlsx_File.Read_xlsx_File("../Data_File/sym_data/NIFTY.xlsx", int(input("0 = NFITY50\n1 = NIFTY100\n2 = NFITY200\n3 = NIFTY-NEXT50\n4 = Please inter your symbols in Port_folio1 and numbers are continues.5 6 7\n Please enter your Choice: ")))
symbols = rf.Read_xlsx_File.symbols(data)
sym = symbols[:5]

In [9]:
yf_dn_obj = dn.yf_dn.yf_data
for symbol in sym:
    data5 = yf_dn_obj(self=symbol, symbol=symbol)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [10]:
data5

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-12-26 09:15:00+05:30,991.599976,993.250000,991.299988,992.950012,992.950012,0
2023-12-26 09:20:00+05:30,992.950012,993.650024,991.549988,993.250000,993.250000,174523
2023-12-26 09:25:00+05:30,993.200012,993.250000,990.500000,991.099976,991.099976,137344
2023-12-26 09:30:00+05:30,991.000000,992.650024,990.849976,992.099976,992.099976,149987
2023-12-26 09:35:00+05:30,992.200012,993.549988,991.500000,991.500000,991.500000,73336
...,...,...,...,...,...,...
2024-02-21 15:05:00+05:30,1053.349976,1054.199951,1052.900024,1053.000000,1053.000000,540042
2024-02-21 15:10:00+05:30,1053.000000,1054.300049,1052.250000,1052.300049,1052.300049,636899
2024-02-21 15:15:00+05:30,1052.349976,1052.650024,1051.449951,1051.699951,1051.699951,620548
